In [2]:
import torch
# import torchvision
import matplotlib.pyplot as plt
import numpy as np
import math
import sys
from scipy.io import savemat, loadmat
from tqdm import tqdm
from torch.nn import DataParallel
import csv
from numba import njit, jit
import pandas as pd

In [3]:
# header = ['epoch', 'channel','value']
# name = "ce"
# folder_header = "/home/sarvagya/Desktop/BCH/"+name+"/"
# csv_file = "data_"+name+".csv"
# temp_mat = "tempmatrix_"+name+".mat"
# with open(folder_header+csv_file, 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(header)
    
# uae_value = {
#     "epoch":[],
#     "channel":[],
#     "value":[]
# }
print(torch.cuda.get_device_name(0))

NVIDIA GeForce GTX 960


In [4]:
# mat = mat73.loadmat(folder_header+temp_mat)
# mat['newmatrix'].shape
# epoch = mat['newmatrix'].shape[0]
# channel = mat['newmatrix'].shape[1]
# matrix_feature = np.zeros([epoch, channel, 58,88])


In [5]:
def energy_map(f_map):
  map = np.zeros((f_map.shape[1], f_map.shape[2]))
  for i in range(f_map.shape[0]):
    map = map + f_map[i]
  map = map/f_map.shape[0]
  return map

@jit
def uae(f_map):
  f_map = f_map[0]
  value=0
  value+=np.sum(f_map)
  return value/(f_map.shape[0]*f_map.shape[1]*f_map.shape[2])

In [6]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16',pretrained=True)
# model = model.

Using cache found in /home/neuroimaging/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/neuroimaging/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 107MB/s]  


In [125]:
layers = [2,4,7,9,12,14,16,19,21,23,26,28,30]
for i in tqdm(range(epoch)):
  for j in range(channel):
    img = mat['newmatrix'][i,j,:,:,:]
    img = np.rot90(img)
    img = img.reshape(-1,img.shape[2],img.shape[0],img.shape[1])
    img = torch.tensor(img, dtype=torch.float32)
    
    value = []
    idx = []
    for k in layers:
      model_test = torch.nn.Sequential(*list(model.children())[0][:k])
      model_test = DataParallel(model_test, device_ids=[0, 1])
      # print(model_test.device_ids)
      img = img.cuda()
      res = model_test(img)
      res = res.cpu().detach().numpy()
      # res = np.squeeze(res)
      value.append(uae(res))
      # res_energy = energy_map(res)
      # matrix_feature[i,j,:,:] = res_energy
    idx = [i,j,value]
    with open(folder_header+csv_file, 'a') as f:
      writer = csv.writer(f)
      writer.writerow(idx)
    value = []
    idx = []
    # uae_value["epoch"].append(i)
    # uae_value["channel"].append(j)
    # uae_value['value'].append(value)

100%|██████████| 167/167 [08:15<00:00,  2.97s/it]


THIS IS TO CREATE AVG MAT FILE FOR UAE VALUES

In [139]:
name = "ce"
folder_header = "/home/sarvagya/Desktop/BCH/"+name+"/"
csv_file = "data_"+name+".csv"
# temp_mat = "tempmatrix_"+name+".mat"
# mat = mat73.loadmat(folder_header+temp_mat)
# mat['newmatrix'].shape
df = pd.read_csv(folder_header+csv_file)
epoch = list(np.unique(list(df['epoch'])))
channel = list(np.unique(list(df['channel'])))

FileNotFoundError: [Errno 2] No such file or directory: '/home/sarvagya/Desktop/BCH/ce/data_ce.csv'

In [140]:
print(len(epoch))
print(len(channel))
# df

100
63


In [135]:
# (df['value'][0].split('[')[1].split(']')[0].split(','))
# uae_avg = df.value.str.split(",",expand=True)
# uae_avg[0] = uae_avg[0].str.split('[', expand=True)[1]
# uae_avg[12] = uae_avg[12].str.split(']', expand=True)[0]
# uae_avg
# elements = df.loc[(df['epoch']==1) & (df['channel']==0)]['value'][0]
# elements = df.iloc[128]['value']
# elements = elements.split('[')[1].split(']')[0].split(',')
# floats = [float(x) for x in elements]
# print(floats)
#64*epoch + channel

In [136]:
avg_uae = {
    'electrode':[],
    'avg_uae':[]
}
for i in tqdm(range(len(channel))):
    value = []
    for j in range(len(epoch)):
        count = len(channel)*j+i
        elements = df.iloc[count]['value']
        elements = elements.split('[')[1].split(']')[0].split(',')
        floats = [float(x) for x in elements]
        value.append(floats)
    avg_uae['electrode'].append(i)
    avg_uae['avg_uae'].append(np.average(value,axis=0))

100%|██████████| 63/63 [00:00<00:00, 190.35it/s]


In [137]:
mat_file = folder_header+"17_uae.mat"
savemat(mat_file, avg_uae)

In [ ]:
#paul 01, db 09, kenny 10, kyle 12, 14 brenden, 15 luke, 17 sm, 
# 19 palmar, 77 harrison, 
#25 ce, 61 perez, 64 jacob, 86 derrin, 89 harrison 2, 91 jasmine, 126 elizabeth,
#127 main, 155 gm, 184 md, 194 ml, 197 js, 200 ps